# Исследовательский анализ данных (EDA) датасета Titanic

Цель анализа: оценить структуру, полноту и качество данных, выявить возможные выбросы и аномалии для последующей подготовки данных к моделированию.


In [1]:
import pandas as pd
import numpy as np

FILE_ID = "1Tj8RQHUh7MTmE88paqDBqDJZ5Ru6mOtL"
file_url = f"https://drive.google.com/uc?export=download&id={FILE_ID}"

# Загрузка датасета с учетом кавычек и пробелов
df_raw = pd.read_csv(file_url, sep=",", quotechar='"', skipinitialspace=True, encoding="utf-8")

print(f"Размер датасета: {df_raw.shape[0]} строк, {df_raw.shape[1]} столбцов")
print("\nПервые 10 строк:")
print(df_raw.head(10))

Размер датасета: 891 строк, 12 столбцов

Первые 10 строк:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   
5            6         0       3   
6            7         0       1   
7            8         0       3   
8            9         1       3   
9           10         1       2   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   
5                                   Moran, Mr. James    male   NaN      0   
6     

In [2]:
df = df_raw.copy()

# Приведение к числовым типам (Int32 nullable и Float32)
int_cols = ["PassengerId", "Survived", "Pclass", "SibSp", "Parch"]
for col in int_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int32")

float_cols = ["Age", "Fare"]
for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce").astype("Float32")

# Текстовые и категориальные признаки
df["Name"] = df["Name"].astype("string")
cat_cols = ["Sex", "Ticket", "Cabin", "Embarked"]
for col in cat_cols:
    df[col] = df[col].astype("string").astype("category")

print("\nТипы данных после приведения:")
print(df.dtypes)


Типы данных после приведения:
PassengerId             Int32
Survived                Int32
Pclass                  Int32
Name           string[python]
Sex                  category
Age                   Float32
SibSp                   Int32
Parch                   Int32
Ticket               category
Fare                  Float32
Cabin                category
Embarked             category
dtype: object


In [3]:
print("\nОбщая информация по датасету:")
df.info()

print("\nКоличество уникальных значений в категориальных столбцах:")
for col in cat_cols:
    unique_vals = df[col].nunique()
    print(f" - {col}: {unique_vals}")

print("\nОсновные статистики числовых характеристик:")
display(df.describe())


Общая информация по датасету:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    Int32   
 1   Survived     891 non-null    Int32   
 2   Pclass       891 non-null    Int32   
 3   Name         891 non-null    string  
 4   Sex          891 non-null    category
 5   Age          714 non-null    Float32 
 6   SibSp        891 non-null    Int32   
 7   Parch        891 non-null    Int32   
 8   Ticket       891 non-null    category
 9   Fare         891 non-null    Float32 
 10  Cabin        204 non-null    category
 11  Embarked     889 non-null    category
dtypes: Float32(2), Int32(5), category(4), string(1)
memory usage: 69.7 KB

Количество уникальных значений в категориальных столбцах:
 - Sex: 2
 - Ticket: 681
 - Cabin: 147
 - Embarked: 3

Основные статистики числовых характеристик:


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.0,891.0,891.0,714.0,891.0,891.0,891.0
mean,446.0,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526498,1.102743,0.806057,49.693432
min,1.0,0.0,1.0,0.42,0.0,0.0,0.0
25%,223.5,0.0,2.0,20.125,0.0,0.0,7.9104
50%,446.0,0.0,3.0,28.0,0.0,0.0,14.4542
75%,668.5,1.0,3.0,38.0,1.0,0.0,31.0
max,891.0,1.0,3.0,80.0,8.0,6.0,512.329224


In [4]:
print("\nПропущенные значения по столбцам:")
missing_counts = df.isna().sum()
missing_percent = (missing_counts / len(df)) * 100
missing_df = pd.DataFrame({"count_missing": missing_counts, "percent_missing": missing_percent})
display(missing_df)

print("Вывод: Колонки с наибольшим процентом пропусков требуют внимания при обработке.")


Пропущенные значения по столбцам:


,count_missing,percent_missing
PassengerId,0,0.000000
Survived,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,177,19.865320
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,0,0.000000


Вывод: Колонки с наибольшим процентом пропусков требуют внимания при обработке.


In [5]:
def find_outliers_iqr(data, col):
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    return outliers

num_cols = int_cols + float_cols
print("\nВыбросы в числовых признаках:")
for col in num_cols:
    outliers = find_outliers_iqr(df, col)
    print(f" - {col}: {len(outliers)} выбросов ({len(outliers)/len(df)*100:.2f} % от данных)")


Выбросы в числовых признаках:
 - PassengerId: 0 выбросов (0.00 % от данных)
 - Survived: 0 выбросов (0.00 % от данных)
 - Pclass: 0 выбросов (0.00 % от данных)
 - SibSp: 46 выбросов (5.16 % от данных)
 - Parch: 213 выбросов (23.91 % от данных)
 - Age: 11 выбросов (1.23 % от данных)
 - Fare: 116 выбросов (13.02 % от данных)


Выводы:
- Датасет содержит 891 строку и 12 столбцов.
- Есть пропуски в столбцах Age, Cabin и Embarked, наибольшие пропуски в Cabin (~77%).
- Выбросы выявлены в числовых столбцах Fare, Age и других; требует дальнейшего анализа и решения.
- Типы данных приведены к оптимальным для дальнейшей работы.

Рекомендуется:
- Обработать пропуски в Age и Embarked, возможно, заменить или удалить.
- Рассмотреть удаление или кодирование Cabin из-за большого количества пропусков.
- Учесть выбросы при построении модели.

In [6]:
# Пример - Создание бинарного признака "IsAlone" по SibSp и Parch
df['IsAlone'] = ((df['SibSp'] + df['Parch']) == 0).astype('Int32')
print("\nДобавлен бинарный признак IsAlone.")
display(df[['SibSp', 'Parch', 'IsAlone']].head())

# Корреляционная матрица числовых признаков (без сложной визуализации)
print("\nКорреляционная матрица:")
corr_matrix = df[num_cols + ['IsAlone']].corr()
display(corr_matrix)


Добавлен бинарный признак IsAlone.


,SibSp,Parch,IsAlone
0,1,0,0
1,1,0,0
2,0,0,1
3,1,0,0
4,0,0,1



Корреляционная матрица:


,PassengerId,Survived,Pclass,SibSp,Parch,Age,Fare,IsAlone
PassengerId,1.000000,-0.005007,-0.035144,-0.057527,-0.001652,0.036847,0.012658,0.057462
Survived,-0.005007,1.000000,-0.338481,-0.035322,0.081629,-0.077221,0.257307,-0.203367
Pclass,-0.035144,-0.338481,1.000000,0.083081,0.018443,-0.369226,-0.549500,0.135207
SibSp,-0.057527,-0.035322,0.083081,1.000000,0.414838,-0.308247,0.159651,-0.584471
Parch,-0.001652,0.081629,0.018443,0.414838,1.000000,-0.189119,0.216225,-0.583398
Age,0.036847,-0.077221,-0.369226,-0.308247,-0.189119,1.000000,0.096067,0.198270
Fare,0.012658,0.257307,-0.549500,0.159651,0.216225,0.096067,1.000000,-0.271832
IsAlone,0.057462,-0.203367,0.135207,-0.584471,-0.583398,0.198270,-0.271832,1.000000
